<a href="https://colab.research.google.com/github/lathifahamirah/practicummis/blob/main/SIDANG_FIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hdbscan

import numpy as np
import pandas as pd
import hdbscan
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
data=pd.read_csv('/content/datacoba.csv', sep=";")
data.head()

In [ ]:
datagizi = data.drop(columns=['NamaProduk'])
datagizi.head()

In [ ]:
from sklearn.manifold import MDS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

X = datagizi.values

# Inisialisasi MDS
mds = MDS(n_components=2, dissimilarity='euclidean', random_state=42, normalized_stress='auto')
X_mds = mds.fit_transform(X)

# Menampilkan nilai stress
print(f'Stress: {mds.stress_:.4f}')

# Menghitung titik tengah (mean atau median) untuk analisis kuadran
x_median = np.median(X_mds[:, 0])
y_median = np.median(X_mds[:, 1])

# Menentukan warna berdasarkan kuadran
colors = []
labels = []
for x, y in zip(X_mds[:, 0], X_mds[:, 1]):
    if x > x_median and y > y_median:
        colors.append('green')  # Kuadran 1
        labels.append('Kuadran 1')
    elif x < x_median and y > y_median:
        colors.append('blue')  # Kuadran 2
        labels.append('Kuadran 2')
    elif x < x_median and y < y_median:
        colors.append('yellow')  # Kuadran 3
        labels.append('Kuadran 3')
    else:
        colors.append('red')  # Kuadran 4
        labels.append('Kuadran 4')

# Visualisasi dengan garis kuadran
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_mds[:, 0], X_mds[:, 1], c=colors, s=50)

# Menambahkan garis kuadran
plt.axhline(y=y_median, color='black', linestyle='--', linewidth=1)
plt.axvline(x=x_median, color='black', linestyle='--', linewidth=1)

# Menambahkan legenda
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='Kuadran 1'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Kuadran 2'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='yellow', markersize=10, label='Kuadran 3'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Kuadran 4')
]
plt.legend(handles=legend_elements, title="Kuadran")

# Label dan judul
plt.title("MDS Visualization with Quadrant Analysis")
plt.xlabel("MDS Dimension 1")
plt.ylabel("MDS Dimension 2")

plt.grid(True)
plt.show()


In [ ]:
# Menyimpan koordinat dan label kuadran ke DataFrame
df_mds = pd.DataFrame(X_mds, columns=['Dimensi1', 'Dimensi2'], index=datagizi.index)
df_mds['Kuadran'] = labels

# Menghitung jumlah produk per kuadran
jumlah_kuadran = df_mds['Kuadran'].value_counts().reset_index()
jumlah_kuadran.columns = ['Kuadran', 'JumlahProduk']

print(jumlah_kuadran)

In [ ]:
datagizi.columns

In [ ]:
# Tentukan kolom yang relevan untuk analisis
columns_of_interest = datagizi.columns

# Tentukan ambang batas tinggi (misalnya nilai lebih dari rata-rata atau nilai tertentu)
high_threshold = datagizi[columns_of_interest].mean()  # Menggunakan rata-rata sebagai ambang batas

# Fungsi untuk mengklasifikasikan produk berdasarkan kandungan tinggi/rendah
def classify_product(row):
    classification = []
    for col in columns_of_interest:
        if row[col] > high_threshold[col]:
            classification.append(f'High {col}')
        else:
            classification.append(f'Low {col}')
    return ', '.join(classification)

# Terapkan fungsi untuk setiap produk dan simpan dalam kolom baru
datagizi['Classification'] = datagizi.apply(classify_product, axis=1)

# Kelompokkan produk berdasarkan klasifikasi mereka
grouped = datagizi.groupby('Classification').size().reset_index(name='Total')

# Menampilkan hasil sesuai format yang diinginkan
for _, row in grouped.iterrows():
    classification = row['Classification']
    total = row['Total']
    product_indexes = datagizi[datagizi['Classification'] == classification].index.tolist()
    print(f"**{classification}** (Total: {total} produk): {', '.join(map(str, product_indexes))}")


In [ ]:
datagizi.columns

In [ ]:
datagizi = data.drop(columns=['NamaProduk','Classification'], errors='ignore')

In [ ]:
datagizi.columns

In [ ]:
# Analisis Deskriptif
nilai_mean = np.mean(datagizi, axis=0)
nilai_max = np.max(datagizi, axis=0)
nilai_min = np.min(datagizi, axis=0)
nilai_std_dev = np.std(datagizi, axis=0)
# Membuat DataFrame dengan menggunakan pandas
df_analisis_deskriptif = pd.DataFrame({
    'Mean': nilai_mean,
    'Max': nilai_max,
    'Min': nilai_min,
    'Std Dev': nilai_std_dev
    })
# Menampilkan DataFrame
print("DataFrame hasil analisis:")
print(df_analisis_deskriptif)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Membuat figure dengan 7 subplot (3 baris x 3 kolom)
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# Flatten axes untuk memudahkan iterasi
axes = axes.flatten()

# List kolom yang akan diplot
columns = ['Karbohidrat', 'LemakTotal', 'LemakJenuh', 'Gula', 'Garam']
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'grey']

# Loop untuk membuat boxplot di setiap subplot
for i, (col, color) in enumerate(zip(columns, colors)):
    sns.boxplot(y=datagizi[col], color=color, ax=axes[i])
    axes[i].set_title(f'Boxplot untuk {col} pada Mi Instan')
    axes[i].set_ylabel('Nilai')
    axes[i].set_xlabel(col)

# Menyembunyikan subplot yang tidak digunakan
for j in range(len(columns), len(axes)):
    axes[j].axis('off')

# Menyimpan plot sebagai file gambar
plt.savefig('boxplot_kandungan_gizi.png', dpi=300, bbox_inches='tight')

# Menampilkan plot
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd

outliers_dict = {}

for kolom in columns:
    Q1 = data[columns].quantile(0.25)
    Q3 = data[columns].quantile(0.75)
    IQR = Q3 - Q1
    batas_bawah = Q1 - 1.5 * IQR
    batas_atas = Q3 + 1.5 * IQR

    # Ambil data yang menjadi outlier
    outliers = data[(data[columns] < batas_bawah) | (data[columns] > batas_atas)]

    # Simpan ke dictionary dengan nama kolom nutrisi
    outliers_dict[kolom] = outliers[['NamaProduk', kolom]]

print("Outlier Karbohidrat:")
print(outliers['Karbohidrat'].dropna())

print("Outlier Lemak Total:")
print(outliers['LemakTotal'].dropna())

print("Outlier Lemak Jenuh:")
print(outliers['LemakJenuh'].dropna())

print("Outlier Gula:")
print(outliers['Gula'].dropna())

print("Outlier Garam:")
print(outliers['Garam'].dropna())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()
# Transformasi data menggunakan MinMaxScaler
gizistd = scaler.fit_transform(datagizi)
# data standar ke dataframe
gizistd_df = pd.DataFrame(gizistd, columns=datagizi.columns)
gizistd_df

In [ ]:
np.random.seed(42)
#Inisialisasi HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=5, metric='euclidean', gen_min_span_tree=True)
# Fit model ke data
clusterer.fit(gizistd_df)

In [ ]:
plt.figure(figsize=(8, 6))
clusterer.minimum_spanning_tree_.plot(edge_cmap='viridis',
edge_alpha=0.6,
node_size=100,
edge_linewidth=2)

In [ ]:
plt.figure(figsize=(8, 6))
clusterer.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

In [ ]:
plt.figure(figsize=(8, 6))
clusterer.condensed_tree_.plot()

In [ ]:
plt.figure(figsize=(8, 6))
clusterer.condensed_tree_.plot(select_clusters=True,
selection_palette=sns.color_palette())

In [ ]:
# Hasil clustering
print(clusterer.labels_)

In [ ]:
print("Cluster Labels:")
print(clusterer.labels_)
print("Cluster Probabilities:")
print(clusterer.probabilities_)

In [ ]:
# Menambahkan kolom 'Cluster' ke DataFrame
gizistd_df['Cluster'] = clusterer.labels_

In [ ]:
# Menampilkan jumlah klaster yang terbentuk
n_clusters = len(gizistd_df['Cluster'].unique())
print(f"\nJumlah Klaster yang Terbentuk: {n_clusters}")

In [ ]:
# Menampilkan jumlah anggota setiap cluster
cluster_member_counts = gizistd_df['Cluster'].value_counts()
print("\nJumlah Anggota Setiap Cluster:")
print(cluster_member_counts)

In [ ]:
# Menampilkan anggota mana saja yang masuk ke dalam setiap cluster
print("\nAnggota Setiap Cluster:")

for cluster_id, member_count in cluster_member_counts.items():
    cluster_members = gizistd_df[gizistd_df['Cluster'] == cluster_id].index.tolist()
    print(f"\nCluster {cluster_id} (Jumlah anggota: {member_count}):\n{cluster_members}")

In [ ]:
# Filter cluster dengan label -1
filtered_labels = clusterer.labels_[clusterer.labels_ != -1]
filtered_probabilities = clusterer.probabilities_[clusterer.labels_ != -1]

In [ ]:
print("Cluster Labels:")
print(filtered_labels)
print("Cluster Probabilities:")
print(filtered_probabilities)

In [ ]:
# Menghapus cluster -1 dari DataFrame
data_std_filtered = gizistd_df.query('Cluster != -1')

# Memastikan bahwa kolom 'Cluster' ada dalam DataFrame sebelum dihapus
if 'Cluster' in data_std_filtered.columns:
    # Drop kolom 'Cluster'
    dataakhir = data_std_filtered.drop(columns=['Cluster'])
else:
    dataakhir = data_std_filtered

# Menampilkan hasil
dataakhir

In [ ]:
data_std_filtered

In [ ]:
# Hitung silhouette score
silhouette = silhouette_score(dataakhir, filtered_labels)
print("Nilai Silhouette Score: ", silhouette)

In [ ]:
from sklearn.metrics import calinski_harabasz_score

# Hitung CH Index
chi = calinski_harabasz_score(dataakhir, filtered_labels)
print(f"Calinski-Harabasz Index: {chi:.2f}")

In [ ]:
data_cluster1 = data_std_filtered[data_std_filtered['Cluster'] == 1]
data_cluster0 = data_std_filtered[data_std_filtered['Cluster'] == 0]
cluster1 = data_cluster1.drop(columns=['Cluster'])
cluster0 = data_cluster0.drop(columns=['Cluster'])

In [ ]:
from scipy.stats import shapiro

clusters = [cluster0, cluster1]
variables = ['LemakTotal', 'LemakJenuh', 'Karbohidrat', 'Gula', 'Garam']

for var in variables:
    for i, cl in enumerate(clusters):
        if var in cl.columns:
            stat, p = shapiro(cl[var].dropna())
            print(f"{var} - Cluster {i}: W={stat:.3f}, p={p:.3f} → {'Normal' if p > 0.05 else 'Tidak Normal'}")

In [ ]:
from scipy.stats import levene

# Uji Homogenitas (Levene's Test)
for var in variables:
    data_cluster0 = cluster0[var]
    data_cluster1 = cluster1[var]
    stat, p_value = levene(data_cluster0, data_cluster1)
    print(f"Homogenitas - {var}:")
    print(f"  Statistik = {stat:.3f}, p-value = {p_value:.3f}")
    print(f"  Kesimpulan: {'Varian Homogen' if p_value > 0.05 else 'Varian Tidak Homogen'}\n")

In [ ]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(cluster1, cluster0, alternative='two-sided')
print("=== Hasil Mann-Whitney U Test ===")
print("Statistik Uji:", stat)
print("p-value:", p)